# Cook Book Recipes – README 😊

## Обзор 🚀

Данный проект представляет собой демонстрационный пример многоагентной системы, предназначенной для автоматизированного создания, проверки и доработки тестовых сценариев и соответствующего им тестового кода. Архитектура построена на использовании языковых моделей (AzureChatOpenAI) в связке с библиотеками для работы с чат-агентами (langchain) и графовыми состояниями (langgraph). Проект демонстрирует принцип кооперации нескольких агентов, каждый из которых выполняет свою специализированную задачу. 👍

## Цель проекта 🎯

Основная задача проекта – автоматизация процесса:
- **Формулирования тестовых сценариев.** Один агент генерирует набор тестовых сценариев для проверки функционала.
- **Валидации сценариев.** Второй агент проверяет сгенерированные сценарии, при необходимости предлагая улучшения (через механизм "REWRITE").
- **Генерации тестового кода.** После утверждения сценариев третий агент пишет код тестов на выбранном языке программирования (в примере — Python).
- **Проверки тестового кода.** Четвёртый агент осуществляет ревизию кода и, если требуется, предлагает доработки.

Таким образом, система позволяет итеративно совершенствовать тестовые сценарии и их реализацию, гарантируя высокое качество финального результата. 😊

## Основные компоненты 🛠

### 1. Импорты и настройка окружения 🌐
- **Библиотеки и зависимости.** Используются стандартные модули для работы с переменными окружения (dotenv, os), а также специализированные библиотеки для интеграции с языковыми моделями и построения графов состояний.
- **Переменные окружения.** Загрузка конфигурации через `.env` и установка переменной `LANGCHAIN_PROJECT` для корректного функционирования.

### 2. Создание агентов 🤖
- **Функция `create_agent`.** Формирует шаблон чата с предустановленным системным сообщением, которое определяет задачу агента.
- **Агенты для написания тестовых сценариев.** Агент `ScenarioWriter` генерирует тестовые сценарии, а `ScenarioReviewer` проверяет их корректность и предлагает доработки.
- **Агенты для работы с кодом.** Агент `CodeWriter` пишет тестовый код на Python, а `CodeReviewer` проводит его ревизию.

### 3. Построение и выполнение графа состояний 🔄
- **StateGraph.** Система использует графовое представление последовательности действий, где узлы соответствуют этапам генерации и проверки тестовых сценариев/кода.
- **Маршрутизация (`router`).** Функция определяет следующий шаг в зависимости от содержимого сообщений (например, если в ответе содержится ключевое слово `REWRITE` или `FINAL ANSWER`).
- **Запуск графа.** Граф последовательно вызывает агентов, обеспечивая итеративный процесс до получения окончательного результата.

### 4. Пример сценария использования 💡
В примере описан тестовый кейс, связанный с безопасной авторизацией пользователя с использованием электронной почты и пароля. Сценарий включает проверку контроля доступа на основе ролей (администратор, пользователь, гость). Агенты генерируют тестовые сценарии и соответствующий тестовый код для реализации данного функционала.

## Как запустить проект 🚀

1. **Настройка окружения.** Создайте файл `.env` с необходимыми переменными, такими как `AZURE_OPENAI_ENDPOINT` и `AZURE_OPENAI_API_VERSION`.
2. **Установка зависимостей.** Убедитесь, что установлены все используемые библиотеки (например, `dotenv`, `langchain`, `langgraph` и другие).
3. **Запуск в Jupyter Lab.** Откройте тетрадку в Jupyter Lab и выполните ячейки последовательно. Результатом будет граф, отображающий последовательность действий агентов, и вывод финальных сообщений с тестовыми сценариями и тестовым кодом. 💻

## Заключение ✨

Данный Cook Book Recipes демонстрирует возможность создания сложных многоагентных систем для автоматизации генерации тестовых сценариев и кода. Архитектура позволяет разделять ответственность между агентами, что облегчает процесс проверки и доработки результатов. Такой подход можно масштабировать и адаптировать для решения различных задач в области автоматизированного тестирования и разработки программного обеспечения. 😊

## Load Env Vars

In [ ]:
# Импорты библиотек
# -----------------------------------------------------------------------------
# Импорты для работы с переменными окружения
from dotenv import load_dotenv
import os

# Импорты для работы с моделями языка
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Импорты для работы с графами состояний
from langgraph.graph import END, StateGraph, START

# Импорты для работы с типами данных
import operator
from typing import Annotated, Sequence, TypedDict, List, Literal

# Импорты для работы с функциями
import functools

# Импорты для работы с отображением графов
from IPython.display import Image, display

# -----------------------------------------------------------------------------

# Загрузка переменных окружения из файла .env
load_dotenv()

# Установка переменной окружения для проекта
os.environ["LANGCHAIN_PROJECT"] = "Multi-Agents-Colab-Architecture"

# Имя развертывания и язык программирования
deployment_name = 'gpt-4o-mini'
code_language   = 'Python'

## Create Agents

In [ ]:
def create_agent(llm, system_message: str) -> ChatPromptTemplate:
    """
    Description:
        Создает агента с заданным системным сообщением.

    Args:
        llm: Модель языка.
        system_message: Системное сообщение для агента.

    Returns:
        Шаблон чата с частично примененным системным сообщением.

    Examples:
        >>> create_agent(llm, "You are a helpful AI assistant.")
        <ChatPromptTemplate object>
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                " Вы полезный AI-ассистент, сотрудничающий с другими ассистентами."
                " Если вы не можете полностью ответить, это нормально, другой ассистент "
                " поможет с того места, где вы остановились. Выполняйте то, что можете, чтобы продвинуться."
                " Помните, что нужно сосредоточиться только на своей задаче, не выполняйте работу другого ассистента."
                " Если у вас или у других ассистентов есть окончательный ответ или результат,"
                " начните свой ответ с FINAL ANSWER, чтобы команда знала, что нужно остановиться."
                " \n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    return prompt.partial(system_message=system_message) | llm

## Define State Message

In [ ]:
class AgentState(TypedDict):
    """
    Description:
        Тип данных для состояния агента.

    Attributes:
        sender: Отправитель сообщения.
        messages: Последовательность сообщений.
    """
    sender: str
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Define Agent Nodes


In [ ]:
def agent_node(state: AgentState, agent: ChatPromptTemplate, name: str) -> AgentState:
    """
    Description:
        Выполняет узел агента и возвращает результат.

    Args:
        state: Состояние агента.
        agent: Агент.
        name: Имя агента.

    Returns:
        Обновленное состояние агента.

    Examples:
        >>> agent_node(state, agent, "AgentName")
        {'messages': [<Message object>], 'sender': 'AgentName'}
    """
    result = agent.invoke(state)
    return {
        "messages": [result],
        "sender": name,
    }

# -----------------------------------------------------------------------------

# Инициализация модели языка
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

# -----------------------------------------------------------------------------

# Создание агентов для написания и проверки тестовых сценариев
test_scenario_writer_agent = create_agent(
    llm=llm,
    system_message="""
        Вы должны предоставить набор связных и хорошо определенных тестовых сценариев для другого агента, чтобы он мог писать коды,
        или улучшать существующие тестовые сценарии в соответствии с данными инструкциями.
        Вы не должны предоставлять коды или другие функции, которые не являются вашей основной задачей.
    """
)
ts_writer_node = functools.partial(agent_node, agent=test_scenario_writer_agent, name="ScenarioWriter")

test_scenario_reviewer_agent = create_agent(
    llm,
    system_message="""
        Вы должны проверить набор тестовых сценариев,
        и если тестовые сценарии нуждаются в улучшении, вы должны начать свой ответ с REWRITE и предоставить инструкции по улучшению.
        Вы не должны предоставлять коды или другие функции, которые не являются вашей основной задачей.
    """,
)
ts_reviewer_node = functools.partial(agent_node, agent=test_scenario_reviewer_agent, name="ScenarioReviewer")

# -----------------------------------------------------------------------------

# Создание агентов для написания и проверки тестового кода
test_code_writer_agent = create_agent(
    llm=llm,
    system_message=f"Вы должны писать код на {code_language} для тестовых сценариев",
)
tc_writer_node = functools.partial(agent_node, agent=test_code_writer_agent, name="CodeWriter")

test_code_reviewer_agent = create_agent(
    llm=llm,
    system_message=f"""
        Вы должны проверить код на {code_language},
        и если код нуждается в улучшении, вы должны начать свой ответ с REWRITE и предоставить инструкции по улучшению.
        Вы не должны предоставлять коды или другие функции, которые не являются вашей основной задачей.
    """,
)
tc_reviewer_node = functools.partial(agent_node, agent=test_code_reviewer_agent, name="CodeReviewer")

## Define Edge Logic

In [ ]:
def router(state: AgentState) -> Literal["__end__", "continue", "rewrite"]:
    """
    Description:
        Определяет маршрут на основе состояния агента.

    Args:
        state: Состояние агента.

    Returns:
        Строка, указывающая маршрут.

    Examples:
        >>> router({"messages": [HumanMessage(content="REWRITE")]})
        'rewrite'
    """
    messages = state["messages"]
    last_message = messages[-1]
    if "REWRITE" in last_message.content:
        return "rewrite"
    if "FINAL ANSWER" in last_message.content:
        return "__end__"
    return "continue"

## Define the Graph

In [ ]:
# Create Workflow
workflow = StateGraph(AgentState)

workflow.add_node("ScenarioWriter", ts_writer_node)
workflow.add_node("ScenarioReviewer", ts_reviewer_node)
workflow.add_node("CodeWriter", tc_writer_node)
workflow.add_node("CodeReviewer", tc_reviewer_node)

workflow.add_conditional_edges(
    "ScenarioWriter",
    router,
    {
        "continue": "ScenarioReviewer",
        "__end__": END
    },
)

workflow.add_conditional_edges(
    "ScenarioReviewer",
    router,
    {
        "continue": "CodeWriter",
        "rewrite": "ScenarioWriter",
        "__end__": "CodeWriter"
    },
)

workflow.add_conditional_edges(
    "CodeWriter",
    router,
    {
        "continue": "CodeReviewer",
        "__end__": END
    },
)

workflow.add_conditional_edges(
    "CodeReviewer",
    router,
    {
        "continue": "ScenarioWriter",
        "rewrite": "CodeWriter",
        "__end__": END
    },
)

workflow.add_edge(START, "ScenarioWriter")
graph = workflow.compile()

In [ ]:
try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    pass

In [ ]:
objective = """
    Как пользователь, я хочу безопасно войти в приложение, используя свой адрес электронной почты и пароль, чтобы получить доступ к своей персонализированной панели управления и функциям.
    Система также должна поддерживать контроль доступа на основе ролей, чтобы ограничить доступ к определенным областям приложения в зависимости от ролей пользователей (например, администратор, пользователь, гость).
"""

events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f" Я хочу, чтобы вы написали 3 тестовых сценария для этой цели: {objective}"
                f" Для каждого тестового сценария я хочу скрипт кода на {code_language}."
                " Как только напишете код, завершите."
            )
        ],
    },
    # Максимальное количество шагов для выполнения в графе
    {"recursion_limit": 150},
)

In [ ]:
# Вывод результатов
for s in events:
    print("#" * 50)
    for k, v in s.items():
        print(f"{k} : \n", s[k]['messages'][-1].content)
    print()